# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium.webdriver.common.keys import Keys
import time
date_format = "%Y/%m/%d"
today = datetime.today().strftime(date_format)
newsDate = today

#a = datetime.strptime('2020/03/22', date_format).date().strftime(date_format)

# 打開瀏覽器
browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")
time.sleep(5)
html = browser.find_element_by_tag_name('html')
html_source = ""
while newsDate == today:
    # 找出的新聞日期還是今天的話，就繼續抓
    # 每個兩秒鐘自動往下滑
    time.sleep(2)
    #browser.execute_script("window.scrollTo(0, 10000);")
    html.send_keys(Keys.END)

    # 取得資料，丟到 BeautifulSoup 解析
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")

    for d in soup.find(class_="part_list_2").find_all('h3'):
        newsDate = datetime.strptime(d.find(class_="date").text.split(' ')[0], date_format).date().strftime(date_format)
        if not (newsDate == today):
            break


newsList = []
soup = BeautifulSoup(html_source, "html5lib")            
for d in soup.find(class_="part_list_2").find_all('h3'):
    newsDate = datetime.strptime(d.find(class_="date").text.split(' ')[0], date_format).date().strftime(date_format)
    if newsDate == today:
        newsList.append(list([d.find(class_="date").text, d.find_all('a')[-1].text]))
        print(d.find(class_="date").text, d.find_all('a')[-1].text)
    else:
        break
# 關閉瀏覽器
browser.quit();
#print(newsList)

2020/03/22 15:52 悠遊付23日登場！限Android手機「一嗶上車」　「線上儲值」再也不用找加值機
2020/03/22 15:50 我國口罩若充足「該優先援助哪國？」　網大推美、日...他卻搖頭
2020/03/22 15:49 家裡涼蓆壞掉了！機靈媽加工「哄小孩神器」獲整把　網笑炸：打到第3代抓周
2020/03/22 15:48 單身男沒自信進一步追求...魏如萱傳授「心機撩妹法」必學！
2020/03/22 15:45 中國大陸稱連3天「境內零確診」　陳時中：姑且聽之，病毒很狡猾
2020/03/22 15:44 川普堅稱「中國病毒」！美國亞裔頻遭攻擊　怒斥：仇恨當武器
2020/03/22 15:41 中途浪貓懶得自己理毛！　「舔手授權」奴才幫洗臉笑翻10萬人
2020/03/22 15:40 酒精國家隊「超前部署」！台酒投入原料生產試驗　原能會纖維酒精技術可協助
2020/03/22 15:40 台首宗養護機構護理師確診！陳時中「半夜奔現場指揮」：全面清空消毒
2020/03/22 15:39 《美味的廣場》防疫時期暖胃又暖心！白種元率希澈、桐俊好療癒
2020/03/22 15:37 影／張上淳兒子出國滑雪惹議！陳時中：沒有違反規定，問題不大
2020/03/22 15:37 中研院4人群聚感染！指揮中心：是老師到美國染病回台傳3人
2020/03/22 15:36 1天沒回家「客廳像強震過後」　媽氣炸放話：7天後辦貓咪追思會
2020/03/22 15:35 不受疫情影響！吳宗憲副業業績反而成長　產品大賣原因曝光
2020/03/22 15:34 錄《RM》偷放屁3次被抓包！「兇手出爐」全場四處逃竄…真實反應全播出
2020/03/22 15:29 若搭機前發燒被拒載怎麼辦？　指揮中心曝SOP：納入居家檢疫
2020/03/22 15:28 《路透》：東奧委員會秘密準備「延期替代方案」　預計3月底討論
2020/03/22 15:27 台灣新冠肺炎累計169名確診！　2死亡、28人解除隔離
2020/03/22 15:25 為掌握對台控制權　吳嘉隆：美國不會讓台灣獨立　
2020/03/22 15:24 養護機構護理師確診！全員急召回檢疫…53住民移出由2醫院協助
2020/03/22 15:21 日職／王柏融炸裂！熱身賽第2轟出爐　

## 2. 取出現在時間兩小時內的新聞

In [10]:
import pytz
from dateutil.relativedelta import relativedelta
jp = pytz.timezone('Asia/Tokyo')
last2HoursDateTime = datetime.now() - timedelta(hours = 2)
jpdt = jp.localize(last2HoursDateTime)
# 取得2個小時前的utc時間點
utc_dt_2hrs = jpdt.astimezone(pytz.utc)

# Function to covert string to datetime 
def convert(date_time): 
    tw = pytz.timezone('Asia/Taipei')
    myformat = "%Y/%m/%d %H:%M" # The format 
    twdt = tw.localize(datetime.strptime(date_time, myformat))
    return twdt.astimezone(pytz.utc) 


for n in newsList:
    # news utc datetime
    utcNewsDT = convert(n[0])
    # news title
    newsTitle = n[1]
    diff = relativedelta(utcNewsDT, utc_dt_2hrs)
    if (diff.hours > 0) or (diff.hours == 0 and diff.minutes >0):
        print(n[0], newsTitle) 



2020/03/22 15:52 悠遊付23日登場！限Android手機「一嗶上車」　「線上儲值」再也不用找加值機
2020/03/22 15:50 我國口罩若充足「該優先援助哪國？」　網大推美、日...他卻搖頭
2020/03/22 15:49 家裡涼蓆壞掉了！機靈媽加工「哄小孩神器」獲整把　網笑炸：打到第3代抓周
2020/03/22 15:48 單身男沒自信進一步追求...魏如萱傳授「心機撩妹法」必學！
2020/03/22 15:45 中國大陸稱連3天「境內零確診」　陳時中：姑且聽之，病毒很狡猾
2020/03/22 15:44 川普堅稱「中國病毒」！美國亞裔頻遭攻擊　怒斥：仇恨當武器
2020/03/22 15:41 中途浪貓懶得自己理毛！　「舔手授權」奴才幫洗臉笑翻10萬人
2020/03/22 15:40 酒精國家隊「超前部署」！台酒投入原料生產試驗　原能會纖維酒精技術可協助
2020/03/22 15:40 台首宗養護機構護理師確診！陳時中「半夜奔現場指揮」：全面清空消毒
2020/03/22 15:39 《美味的廣場》防疫時期暖胃又暖心！白種元率希澈、桐俊好療癒
2020/03/22 15:37 影／張上淳兒子出國滑雪惹議！陳時中：沒有違反規定，問題不大
2020/03/22 15:37 中研院4人群聚感染！指揮中心：是老師到美國染病回台傳3人
2020/03/22 15:36 1天沒回家「客廳像強震過後」　媽氣炸放話：7天後辦貓咪追思會
2020/03/22 15:35 不受疫情影響！吳宗憲副業業績反而成長　產品大賣原因曝光
2020/03/22 15:34 錄《RM》偷放屁3次被抓包！「兇手出爐」全場四處逃竄…真實反應全播出
2020/03/22 15:29 若搭機前發燒被拒載怎麼辦？　指揮中心曝SOP：納入居家檢疫
2020/03/22 15:28 《路透》：東奧委員會秘密準備「延期替代方案」　預計3月底討論
2020/03/22 15:27 台灣新冠肺炎累計169名確診！　2死亡、28人解除隔離
2020/03/22 15:25 為掌握對台控制權　吳嘉隆：美國不會讓台灣獨立　
2020/03/22 15:24 養護機構護理師確診！全員急召回檢疫…53住民移出由2醫院協助
2020/03/22 15:21 日職／王柏融炸裂！熱身賽第2轟出爐　

## 3. 根據範例，取出三天前(日本時間)下午三點到五點的新聞

In [11]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains


def convertJP2Utc(date_time): 
    jp = pytz.timezone('Asia/Tokyo')
    myformat = "%Y/%m/%d %H:%M" # The format 
    jpdt = jp.localize(datetime.strptime(date_time, myformat))
    return jpdt.astimezone(pytz.utc) 

# 3天前(執行時間是日本時間)
last3DaysDateTime = datetime.now() - timedelta(days = 3)
jpdt = jp.localize(last3DaysDateTime)
utc_dt_3days = jpdt.astimezone(pytz.utc)
# 3天前下午三點到五點
last3DaysDT_15 = last3DaysDateTime.strftime("%Y/%m/%d 15:00")
last3DaysDT_17 = last3DaysDateTime.strftime("%Y/%m/%d 17:00")
#jpdt = jp.localize(last3DaysDT_15)
# 取得三天前下午三點的utc時間點
utc_dt_3days_15 = convertJP2Utc(last3DaysDT_15)
# 取得2個小時前的utc時間點
utc_dt_3days_17 = convertJP2Utc(last3DaysDT_17)


#確認新聞發佈時間介於指定的時間區間
def checkIfInTimePeriod(startTime, endTime, srhTime):
    if startTime < endTime:
        return startTime <= srhTime <= endTime
    else: #Over midnight
        return srhTime >= startTime or srhTime <= endTime

#關閉搜尋資訊flag
edFlg = False
# 打開瀏覽器
browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")
#進入頁面後先等待10秒
time.sleep(10)

#搜尋年
selectYear = Select(browser.find_element_by_id("selY"))
selectYear.select_by_value(str(last3DaysDateTime.year))
#搜尋月
selectMonth = Select(browser.find_element_by_id("selM"))
selectMonth.select_by_value(str(last3DaysDateTime.month))
#搜尋日
selectDay = Select(browser.find_element_by_id("selD"))
selectDay.select_by_value(str(last3DaysDateTime.day))

element = browser.find_element_by_id("button")
actions = ActionChains(browser)
#actions.move_to_element(element).perform() #適用於hyperlink
browser.execute_script("arguments[0].click();", element) 
#browser.find_element_by_id('button').click()
#搜尋結束，並進入頁面後先等待10秒
time.sleep(10)
html = browser.find_element_by_tag_name('html')
while (not edFlg):
    # 找出的新聞日期還是今天的話，就繼續抓
    # 每個5秒鐘自動往下滑
    time.sleep(5)
    
    #browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html.send_keys(Keys.END)

    # 取得資料，丟到 BeautifulSoup 解析
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    lstNews = soup.find(class_="part_list_2").find_all('h3')
    utcNewsDT = convert(lstNews[len(lstNews)-1].find(class_="date").text)
    # 下午三點以前的資料不在搜尋範圍中
    if utcNewsDT < utc_dt_3days_15:
        edFlg = True
                     
for d in soup.find(class_="part_list_2").find_all('h3'):
    utcNewsDT = convert(d.find(class_="date").text)
    # 僅抽取出下午三點到五點之間的新聞
    if(checkIfInTimePeriod(utc_dt_3days_15,utc_dt_3days_17,utcNewsDT) == True):
        print(d.find(class_="date").text, d.find_all('a')[-1].text)

# 關閉瀏覽器
browser.quit();

2020/03/19 16:00 NBA停賽正面影響全美國　主席：所有人開始重視防疫　
2020/03/19 16:00 「悄悄的我出國⋯居家隔離14天」誠品小編致敬文豪名言勸防疫　網讚超有才！
2020/03/19 16:00 香港40年連鎖店撐不下去　大眾書局16家分店今起結束營業
2020/03/19 15:59 《李屍》朱智勛震驚她出演！　從小就是全智賢粉絲「根本沒見到」
2020/03/19 15:59 最無須囤的就是食物！　律師揭「疫期10項必囤物」：我們都會再好好地起來
2020/03/19 15:58 李運慶酒醉被洋妞「打包帶走」！　在「陌生房間」驚醒…光腳逃跑急call朋友求救
2020/03/19 15:57 中華醫大職員下月逆勢調薪6％　員工振奮感謝
2020/03/19 15:57 以為同事離婚…約她午休聊天變車震！男誤上人妻慘賠10萬
2020/03/19 15:55 日本前首相曾接觸確診官員　尚未檢疫無相關症狀
2020/03/19 15:55 與確診同學共餐後「發燒37.8°C」！南部國立大學2生篩檢結果出爐　校方說明了
2020/03/19 15:54 台東森林公園東南側日前火災　自即日起補植苗木7千多棵
2020/03/19 15:54 台指期大跌566點　分析師：8800點換倉缺口難回補、震盪整理機會大
2020/03/19 15:53 網購口罩記得收電子信   「唐鳳有寄匯款方式給大家」
2020/03/19 15:53 藍名嘴「暗指」台死亡人數不尋常？楊文嘉轟：販賣恐懼
2020/03/19 15:52 喝起來！迷客夏新開板橋車站店、Sharetea回歸東區通通限時「買1送1」
2020/03/19 15:51 元大期營運再成長　發放現金股利3.2元創歷史新高
2020/03/19 15:51 因應台積電員工確診案例　鴻海在台啟動全球防疫機制
2020/03/19 15:50 馮迪索《血衛》上映7天「被迫下片」　片商宣布「放到網路」：在家裡看
2020/03/19 15:50 自私女確診新冠肺炎還想玩！　帶病毒「爽躺小島沙灘」繼續度假
2020/03/19 15:49 王子激動演戲像馬景濤！黃仲崑笑爆料：工作人員要保險了
2020/03/19 15:49 口罩2.0首波預購「開始付款」　指揮中心公佈下一波預購時間
2020/0